In [ ]:
import os
import datetime
import glob
import pandas as pd

In [ ]:
start_note_files = glob.glob("../tmp/timestamps/split_pf/*_started*")

start_notes_stratified_by_tool = {}

# group timestamps based on the alignment tool and searching mode
for start_note in start_note_files:
    file_name = os.path.basename(start_note)
    tool = file_name.split("_split_ag_split_")[0]
    search_type = file_name.split("_split_ag_split_")[1].split("_")[0]
    search_start_notes = start_notes_stratified_by_tool.get((tool, search_type), [])
    search_start_notes.append(start_note)
    start_notes_stratified_by_tool[(tool, search_type)] = search_start_notes

In [ ]:
search_run_time_dict = {}
for search_approach, start_time_stamps in start_notes_stratified_by_tool.items():
    time_taken = 0
    for file in start_time_stamps:
        start_time = datetime.datetime.fromtimestamp(os.path.getmtime(file))
        end_time = datetime.datetime.fromtimestamp(os.path.getmtime(file.replace("started", "ended")))
        run_time = end_time - start_time
        time_taken += run_time.seconds
    search_run_time_dict[search_approach] = time_taken

In [ ]:
tools_full_names = {('fs','pref'): 'Foldseek', 
                    ('fs','exh'): 'Foldseek, without prefilter', 
                    ('hmm', 'pref'): "HMMER", 
                    ('hmm', 'exh'): "HMMER, without prefilter", 
                    ('mm', 'pref'): "MMseqs", 
                    ('mm', 'exh'): "MMseqs, without prefilter",
                    ('rs', 'fast'): "Reseek (fast)", 
                    ('rs', 'sens'): "Reseek, sensitive",  
                    ('rs', 'exh'): "Reseek, without prefilter",
                    ('tm', 'exh'): "TM-align"}

In [ ]:
with open("../data/processed/run_time.tsv", 'w') as file:
    file.write("Search method\tRuntime\n")
    for search_type, search_time in search_run_time_dict.items():
        file.write(f"{tools_full_names[search_type]}\t{search_time/60:.2f}\n")

In [ ]:
search_run_time_dict